In [ ]:

import os
import pandas as pd

from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import os
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException


from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from concurrent.futures import ThreadPoolExecutor
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()

In [ ]:
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")
opt.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])
# opt.add_argument("--headless")
# opt.add_argument("--lang=en")

In [ ]:
def scrape_each_team_players(team_class_list):
    """ scrape each team players """
    driver = webdriver.Chrome(options=opt)
    driver.get(team_class_list[-1])
    df_player=pd.DataFrame()
    try:
        wait_for__player_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, './/table[@class="table box table-striped"]'))
        )
        sleep(1)
        
        player_table=driver.find_element(by=By.XPATH, value='.//table[@class="table box table-striped"]')
        player_table
        table_html = player_table.get_attribute('outerHTML')

        # Convert HTML table to DataFrame
        df_player = pd.read_html(table_html)[0]
        rows = player_table.find_elements(By.XPATH, ".//tbody/tr")

        # Initialize an empty list to store the counts of full stars
        full_star_counts = []

        # Loop through each row
        for row in rows:
            # Find the last column in each row (rating column)
            rating_column = row.find_element(By.XPATH, ".//td[last()]")
            # Count the number of full stars (icon-star-full elements)
            full_star_count = rating_column.find_elements(By.XPATH, ".//i[@class='icon-star-full']")
            # Append the count to the list
            full_star_counts.append(len(full_star_count))
        df_player['Rating'] = full_star_counts
        
        
        df_player["team_name"]=team_class_list[0]
        df_player["year"]=team_class_list[1]
        
        
    except:
#         no table found
        pass
    driver.close()

    return df_player

In [ ]:
def scrape_single_club_url(
        club_url:str,
        driver:object,
        gender='boys'
        ):
    base_link=club_url
    driver=driver
    driver.get(base_link)
    print(f"scrpaing url: {club_url}")
    try:
        wait_for_tables = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, './/table[@class="table table-striped box"]'))
        )
        sleep(1)
        results=driver.find_elements(by=By.XPATH, value='.//table[@class="table table-striped box"]')
        for each_table in results:
            header = each_table.find_element(By.XPATH, './/tr[1]/th[1]').text
            # find specific gender table 
            if gender.lower()  in header.lower():
                #  get all team names, its links and class
                team_names_cells=each_table.find_elements(By.XPATH, './/tr/td[1]')
                team_names = [team.text for team in team_names_cells]
                team_class_cells=each_table.find_elements(By.XPATH, './/tr/td[2]')
                team_classes = [class_.text for class_ in team_class_cells]
                hrefs = each_table.find_elements(By.XPATH, './/tr/td[1]//a[@href]')
                # Extract href attributes and store them in a list of teams 
                href_values = [href.get_attribute('href') for href in hrefs]
                merged_list = list(zip(team_names, team_classes, href_values))
                list_of_lists_teams_n_class = [list(t) for t in merged_list]
                
                # scrape for each team in parallel

                print(f"scrapping for each team in parallel for   {str(len(list_of_lists_teams_n_class))} teams")
                with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust max_workers as needed
                    # Use executor.map to apply the function to each URL in parallel
                    result_dfs = executor.map(scrape_each_team_players, list_of_lists_teams_n_class)
                concatenated_df = pd.concat(result_dfs, ignore_index=True)
                
                print('success ... ')
                return concatenated_df
    except TimeoutException as error:
        print('Caught Timout ' + repr(error))
        # scrape_single_club_url(club_url=club_url,driver=driver,gender=gender)

    except Exception as error:
        print('Caught this error: ' + repr(error))
        return None

#### __main__

In [ ]:
team_gender="boys"
data_folder_path = "../data/extra/insidelacrosse_clubs_teams/"
input_file_name="extracted_clubs_years_2017_2024.xlsx"
input_file_path =os.path.join(os.path.join(data_folder_path,'input',input_file_name))
output_folder_name=f"clubs_teams_{team_gender}"
output_folder_dir = os.path.join(data_folder_path,'output',output_folder_name)
if (not os.path.exists(output_folder_dir)):
    os.mkdir(output_folder_dir)
    print('folder created: '+str(output_folder_dir))
input_df=pd.read_excel(input_file_path)

In [ ]:
update_inputfile_flag=True
starting_index=1000
driver = webdriver.Chrome(options=opt)
for index,row in input_df.iterrows():
    if row['Status'] not in ['Yes','No'] and index > starting_index:
        print("at index: "+str(index))
        result =scrape_single_club_url(club_url=row['Link'],driver=driver,gender='boys')
        if result is None:
            print("set status to No")
            input_df.iat[index,3]='No'
        else:
            result.to_csv(f'{output_folder_dir}/{row["Club Name"].replace(" ","_")}_{str(row["Year"])}.csv')
            print("set status to Yes")
            input_df.iat[index,3]='Yes'
        if update_inputfile_flag:
            if index %10 ==0:   # update excel files after every 10 urls processed
                input_df.to_excel(input_file_path,index=False)
                print("input sheet updated at index: "+str(index))

print("completed....")
driver.close()
